In [6]:
from __future__ import division, print_function
import numpy as np

In [10]:
# ca, mg, so, na, cl, hco, alkalinity

source = [30, 17, 12, 5, 7, 252] # 207
target = [100, 5, 50, 35, 60, 265] # 217

In [21]:
mash_volume = 5.5

# chalk CaCO3
# baking soda NaHCO3
# gypsum CaSO4
# calcium chloride CaCl2
# epsom salt MgSO4
# canning salt NaCl

initial_adjustments = np.array([.5, .5, 1.6, 2.2, 0, .5])

# estimated output
expected_out = np.array([81, 17, 55, 21, 72, 284]) # 233

In [22]:
def adjust_water(source_water, salts, mash_volume):
    adjCa = 0
    adjMg = 0
    adjSO4 = 0
    adjNa = 0
    adjCl = 0
    adjHCO3 = 0
    
    CaCO3 = salts[0] / 2
    NaHCO3 = salts[1]
    CaSO4 = salts[2]
    CaCl2 = salts[3]
    MgSO4 = salts[4]
    NaCl = salts[5]
    
    if CaCO3  > 0:
        adjCa += ((105 * CaCO3) / mash_volume)
        adjHCO3 += ((321 * CaCO3) / mash_volume)
    if NaHCO3 > 0:
        adjNa = adjNa + ((75 * NaHCO3) / mash_volume);
        adjHCO3 = adjHCO3 + ((191 * NaHCO3) / mash_volume)
    if CaSO4 > 0:
        adjCa = adjCa + ((61.5 * CaSO4) / mash_volume);
        adjSO4 = adjSO4 + ((147.4 * CaSO4) / mash_volume)
    if CaCl2 > 0:
        adjCa = adjCa + ((72 * CaCl2) / mash_volume);
        adjCl = adjCl + ((127 * CaCl2) / mash_volume)
    if MgSO4 > 0:
        adjMg = adjMg + ((26 * MgSO4) / mash_volume);
        adjSO4 = adjSO4 + ((103 * MgSO4) / mash_volume)
    if NaCl > 0:
        adjNa = adjNa + ((104 * NaCl) / mash_volume);
        adjCl = adjCl + ((160 * NaCl) / mash_volume)
    
    return np.array([adjCa, adjMg, adjSO4, adjNa, adjCl, adjHCO3]) + source_water
    
adjust_water(source, initial_adjustments, mash_volume)

array([  81.46363636,   17.        ,   54.88      ,   21.27272727,
         72.34545455,  283.95454545])

In [28]:
from scipy.optimize import minimize
from scipy.spatial.distance import euclidean

In [29]:
def to_optimize(source_water, target_water, mash_volume):
    def f(x):
        adjusted_water = adjust_water(source_water, x, mash_volume)
        return euclidean(adjusted_water, target_water)
    return f

In [36]:
res = minimize(to_optimize(source, target, mash_volume), initial_adjustments)
res

      fun: 32.065789859694725
 hess_inv: array([[  5.56213775e-02,  -3.00115586e-02,   1.00650743e-02,
         -3.22655558e-02,  -1.95928337e-04,   1.99052614e-02],
       [ -3.00115586e-02,   2.52732613e-02,  -2.12226341e-03,
          1.19919099e-02,  -1.23291127e-05,  -1.21397055e-02],
       [  1.00650743e-02,  -2.12226341e-03,   4.13817077e-02,
         -1.90201344e-02,   2.01076204e-04,   1.11549208e-02],
       [ -3.22655558e-02,   1.19919099e-02,  -1.90201344e-02,
          1.10745819e-01,  -1.27621051e-04,  -6.41900424e-02],
       [ -1.95928337e-04,  -1.23291127e-05,   2.01076204e-04,
         -1.27621051e-04,   1.38332275e-04,  -1.32332690e-05],
       [  1.99052614e-02,  -1.21397055e-02,   1.11549208e-02,
         -6.41900424e-02,  -1.32332690e-05,   6.50973546e-02]])
      jac: array([  0.00000000e+00,  -4.76837158e-07,   4.76837158e-07,
         9.53674316e-07,   0.00000000e+00,  -4.76837158e-07])
  message: 'Optimization terminated successfully.'
     nfev: 536
      ni

In [39]:
initial_adjustments

array([ 0.5,  0.5,  1.6,  2.2,  0. ,  0.5])

In [41]:
final_adjustments = res.x.round(1)
final_adjustments

array([-0. ,  0.6,  1.7,  2.4, -0. ,  0.3])

In [42]:
euclidean(initial_adjustments, final_adjustments)

0.5916079783099615

In [49]:
print('Initial Salts Norm:', np.linalg.norm(adjust_water(source, initial_adjustments, mash_volume)))
print('Final Salts Norm:', np.linalg.norm(adjust_water(source, final_adjustments, mash_volume)))
print('Difference:', euclidean(
    adjust_water(source, initial_adjustments, mash_volume),
    adjust_water(source, final_adjustments, mash_volume)
))

Initial Salts Norm: 310.247830053
Final Salts Norm: 299.879226869
Difference: 11.796525752725286


310.24783005347928

299.87922686881461